***Master's in Health Data Science (MHEDAS)***

***Medical Imaging***

Professor: Roser Sala Llonch

Year 2025-2026


---



# P3. Functional MRI

In sthis practical session, we will see different types of functional MRI data and how to analyse them

In [ ]:
# Install nilearn
try:
    import nilearn
except ImportError:
    # if not, install it using pip
    !pip install nilearn

##1. The Haxby dataset
A study of visual object category identification. It has been widely used in cognitive neuroscience and for brain decoding.
The original paper can be found [HERE](https://www.science.org/doi/10.1126/science.1063736)



###1.1. Show stimuli
Visualize the set of stimuli used in the task. This is not related to the fMRI data, but it's a good visualization practice.

In [ ]:
import matplotlib.pyplot as plt

from nilearn import datasets
from nilearn.plotting import show

haxby_dataset = datasets.fetch_haxby(subjects=[], fetch_stimuli=True)
stimulus_information = haxby_dataset.stimuli

for stim_type in stimulus_information:
    # skip control images, there are too many
    if stim_type != 'controls':

        file_names = stimulus_information[stim_type]

        fig, axes = plt.subplots(6, 8)
        fig.suptitle(stim_type)

        for img_path, ax in zip(file_names, axes.ravel()):
            ax.imshow(plt.imread(img_path), cmap=plt.cm.gray)

        for ax in axes.ravel():
            ax.axis("off")

show()

**Exercise 1.1.** Take a look at the paper and explain with your words how are these images used in the context of fMRI

*PLACE YOUR ANSWER HERE*

###1.2. Fetch the dataset


Dowload the dataset from nilearn:

In [ ]:
import numpy as np # always good to have numpy and pandas in your environment
import pandas as pd
from nilearn import datasets

# fetch the data (by default, 2nd subject will be fetched, check the documentation if you want to change the subject)
haxby_dataset = datasets.fetch_haxby()

**Exercise 1.2.1** Examine the data that you've just downloaded. Try to identify the different elements in the dataset (don't worry, just make a guess)

In [ ]:
# place your code (if you need any) here


*PLACE YOUR ANSWER HERE*

Get the behavioral data (information about how the subject performed the task):

In [ ]:
# Load target information as string and give a numerical identifier to each
behavioral = pd.read_csv(haxby_dataset.session_target[0], sep=' ')
conditions = behavioral['labels'].values

# Record these as an array of sessions
sessions = behavioral['chunks'].values
unique_sessions = behavioral['chunks'].unique()

# fMRI data: a unique file for each session
func_filename = haxby_dataset.func[0]

**Exercise 1.2.2** Examine what you've loaded.

In [ ]:
# PUT YOUR CODE (IF NEEDED)

*PLACE YOUR ANSWER HERE*

From the session data, we'll create an *event structure* for each session.

In [ ]:
TR = 2.5 # we need to indicate the Time of Repetition (TR) manually
events = {}
# events will take  the form of a dictionary of Dataframes, one per session
for session in unique_sessions:
    # get the condition label per session
    conditions_session = conditions[sessions == session]
    # get the number of scans per session, then the corresponding
    # vector of frame times
    n_scans = len(conditions_session)
    frame_times = TR * np.arange(n_scans)
    # each event last the full TR
    duration = TR * np.ones(n_scans)
    # Define the events object
    events_ = pd.DataFrame(
        {'onset': frame_times, 'trial_type': conditions_session, 'duration': duration})
    # remove the rest condition and insert into the dictionary
    events[session] = events_[events_.trial_type != 'rest']

**Exercise 1.2.3** Take a look at the new DataFrame. The events can be visualized using the 'plot_event' function.



In [ ]:
events

In [ ]:
from nilearn.plotting import plot_event

# PLACE YOUR CODE HERE (tip: look at the help of the function)

### 1.3. Define the GLM model and run the analyses.

We'll use tools from nilearn to generate the statistical model.

REMEMBER: A first level model is the statistical comparison between the different *conditions* (or blocks) from a *single subject*.

You can find more information [HERE](https://nilearn.github.io/stable/glm/index.html)

In [ ]:
z_maps = []
conditions_label = []
session_label = []

# Instantiate the glm
from nilearn.glm.first_level import FirstLevelModel
glm = FirstLevelModel(t_r=TR,
                      mask_img=haxby_dataset.mask,
                      high_pass=.008,
                      smoothing_fwhm=4,
                      memory='nilearn_cache')



Before running the GLM massively, let's try to understand it with one example:




In [ ]:
events[session].trial_type.unique()
from nilearn.image import index_img
fmri_session = index_img(func_filename, sessions ==0)
glm.fit(fmri_session, events = events[0])

In [ ]:
from nilearn.plotting import plot_design_matrix
design_matrix = glm.design_matrices_[0]
plot_design_matrix(design_matrix)
plt.show()

Now, run the GLM on **each** session from the *events* dataframe.  

In [ ]:
# now run the GLM on each session
events[session].trial_type.unique()
from nilearn.image import index_img
for session in unique_sessions:
    # grab the fmri data for that particular session
    fmri_session = index_img(func_filename, sessions == session)

    # fit the glm
    glm.fit(fmri_session, events=events[session])

    # set up contrasts: one per condition
    conditions = events[session].trial_type.unique()
    for condition_ in conditions:
        z_maps.append(glm.compute_contrast(condition_))
        conditions_label.append(condition_)
        session_label.append(session)

**Exercise 1.3.1**: Explain what do you interpret from the figure. You can plot other sessions to see the differences.

###1.4. Understanding the **contrasts** and generate a report of the results.

We'll use a specific function that generates a summary report of the results of the First Level analysis.

In [ ]:
from nilearn.image import mean_img
from nilearn.reporting import make_glm_report
mean_img_ = mean_img(func_filename)
report = make_glm_report(glm,
                         contrasts=conditions,
                         bg_img=mean_img_,
                         )

report  # This report can be viewed in a notebook

###1.5. A decoding pipeline

This part uses Machine Learning to implement a decoder.  

In [ ]:
from nilearn.decoding import Decoder
from sklearn.model_selection import LeaveOneGroupOut
decoder = Decoder(estimator='svc', mask=haxby_dataset.mask, standardize=False,
                  screening_percentile=5, cv=LeaveOneGroupOut())
decoder.fit(z_maps, conditions_label, groups=session_label)

# Return the corresponding mean prediction accuracy compared to chance

classification_accuracy = np.mean(list(decoder.cv_scores_.values()))
chance_level = 1. / len(np.unique(conditions))
print('Classification accuracy: {:.4f} / Chance level: {}'.format(
    classification_accuracy, chance_level))

**Exercise 1.5.1**: examine the inputs and outputs of the decoder and interpret the results.